# Agentic Workflows

Goal is to build an agentic system that generates a short research report through planning, external tool usage, and feedback integration. The workflow will involve:

### Agents

* **Planning Agent / Writer**: Creates an outline and coordinates tasks.
* **Research Agent**: Gathers external information using tools like Arxiv, Tavily, and Wikipedia.
* **Editor Agent**: Reflects on the report and provides suggestions for improvement.


### Research Tools

By importing `research_tools`, you gain access to several search utilities:

- `research_tools.arxiv_search_tool(query)` → search academic papers from **arXiv**  

  *Example:* `research_tools.arxiv_search_tool("neural networks for climate modeling")`

- `research_tools.tavily_search_tool(query)` → perform web searches with the **Tavily API**  

  *Example:* `research_tools.tavily_search_tool("latest trends in sunglasses fashion")`

- `research_tools.wikipedia_search_tool(query)` → retrieve summaries from **Wikipedia**  

  *Example:* `research_tools.wikipedia_search_tool("Ensemble Kalman Filter")`

Run the cell below to make them available.

In [ ]:
# =========================
# Imports
# =========================

# --- Standard library
from datetime import datetime
import re
import json
import ast


# --- Third-party ---
from IPython.display import Markdown, display
from aisuite import Client

# --- Local / project ---
import research_tools

In [ ]:
import unittests

### Initialize client

Create a shared client instance for upcoming calls.

In [ ]:
CLIENT = Client()

## Exercise 1: planner_agent

### Objective
Correctly set up a call to a language model (LLM) to generate a research plan.

### Instructions

1. **Focus Areas**:
   - Ensure `CLIENT.chat.completions.create` is correctly configured.
   - Pass the `model` and `messages` parameters correctly:
     - **Model**: Use `"openai:o4-mini"` by default.
     - **Messages**: Set with `{"role": "user", "content": user_prompt}`.
     - **Temperature**: Fixed at 1 for creative outputs.

### Notes

- The prompt is pre-defined and guides the LLM on task requirements.
- Only return a formatted list of steps — no extra text.

Focus on the LLM call setup to complete the task.

In [ ]:
# planner_agent

def planner_agent(topic: str, model: str = "openai:o4-mini") -> list[str]:
    """
    Generates a plan as a Python list of steps (strings) for a research workflow.

    Args:
        topic (str): Research topic to investigate.
        model (str): Language model to use.

    Returns:
        List[str]: A list of executable step strings.
    """


    # Build the user prompt
    user_prompt = f"""
    You are a planning agent responsible for organizing a research workflow with multiple intelligent agents.

    🧠 Available agents:
    - A research agent who can search the web, Wikipedia, and arXiv.
    - A writer agent who can draft research summaries.
    - An editor agent who can reflect and revise the drafts.

    🎯 Your job is to write a clear, step-by-step research plan **as a valid Python list**, where each step is a string.
    Each step should be atomic, executable, and must rely only on the capabilities of the above agents.

    🚫 DO NOT include irrelevant tasks like "create CSV", "set up a repo", "install packages", etc.
    ✅ DO include real research-related tasks (e.g., search, summarize, draft, revise).
    ✅ DO assume tool use is available.
    ✅ DO NOT include explanation text — return ONLY the Python list.
    ✅ The final step should be to generate a Markdown document containing the complete research report.

    Topic: "{topic}"
    """

    # Add the user prompt to the messages list
    messages = [{"role": "user", "content": user_prompt}]


    # Call the LLM
    response = CLIENT.chat.completions.create(
        # Pass in the model
        model=model,
        # Define the messages. Remember this is meant to be a user prompt!
        messages=messages,
        # Keep responses creative
        temperature=1,
    )


    # Extract message from response
    steps_str = response.choices[0].message.content.strip()

    # Parse steps
    steps = ast.literal_eval(steps_str)

    return steps

In [ ]:
# Test your code!
unittests.test_planner_agent(planner_agent)

 All tests passed!


## Exercise 2: research_agent

### Objective
Set up a call to a language model (LLM) to perform a research task using various tools.

### Instructions

**Focus Areas**:

- **Creating a Custom Prompt**:
  - **Define the Role**: Clearly specify the role, such as "research assistant."
  - **List Available Tools** (as strings inside the prompt, not the actual functions):
    - Use `arxiv_tool` to find academic papers.
    - Use `tavily_tool` for general web searches.
    - Use `wikipedia_tool` for accessing encyclopedic knowledge.
  - **Specify the Task**: Include a placeholder in your prompt for defining the specific task that needs to be accomplished.
  - **Include Date Information**: Add a placeholder for the current date or time to provide context.

- **Creating Messages Dict**:
  - Ensure the `messages` are correctly set with `{"role": "user", "content": prompt}`.

- **Creating Tools List**:
  - Create a list of tools for use, such as `research_tools.arxiv_search_tool`, `research_tools.tavily_search_tool`, and `research_tools.wikipedia_search_tool`.

- **Correctly Setting the Call to the LLM**:
  - Pass the `model`, `messages`, and `tools` parameters accurately.
  - Set `tool_choice` to `"auto"` for automatic tool selection.
  - Limit interactions with `max_turns=6`.

### Notes

- The function provides pre-coded blocks where you need to replace placeholder values.
- The approach allows the LLM to use tools dynamically based on the task.

Focus on accurately setting the messages, tools, and LLM call parameters to complete the task.

In [ ]:
# research_agent

def research_agent(task: str, model: str = "openai:gpt-4o", return_messages: bool = False):
    """
    Executes a research task using tools via aisuite (no manual loop).
    Returns either the assistant text, or (text, messages) if return_messages=True.
    """
    print("==================================")
    print("🔍 Research Agent")
    print("==================================")

    current_time = datetime.now().strftime('%Y-%m-%d')

    # Create a customizable prompt by defining the role (e.g., "research assistant"),
    # listing tools (arxiv_tool, tavily_tool, wikipedia_tool) for various searches,
    # specifying the task with a placeholder, and including a current_time placeholder.
    prompt = f"""

    You are a research assistant that will gather external information using tools like Arxiv, Tavily, and Wikipedia.

    Your goal:
    1. Examine the task {task} to accomplish.
    2. Go through the list of tools available to perform the task.
    3. Once your analysis is done, provide your findings for task {task} and
    include also the current date or time {current_time} to provide context.


    Tools that are available for you to perform the search:
    - arxiv_tool: To find academic papers.
    - tavily_tool: To do general web searches.
    - wikipedia_tool: To access encyclopedic knowledge.

    DO NOT forget to include references where you gather the information from your research!

    """
    # Create the messages dict to pass to the LLM. Remember this is a user prompt!
    messages = [{"role": "user", "content": prompt}]

    # Save all of your available tools in the tools list. These can be found in the research_tools module.
    # You can identify each tool in your list like this:
    # research_tools.<name_of_tool>, where <name_of_tool> is replaced with the function name of the tool.
    tools = []

    # Call the model with tools enabled
    response = CLIENT.chat.completions.create(
        # Set the model
        model=model,
        # Pass in the messages. You already defined this!
        messages=messages,
        # Pass in the tools list. You already defined this!
        tools=tools,
        # Set the LLM to automatically choose the tools
        tool_choice="auto",
        # Set the max turns to 6
        max_turns=6
    )


    content = response.choices[0].message.content
    print("✅ Output:\n", content)


    return (content, messages) if return_messages else content

In [ ]:
# Test your code!
unittests.test_research_agent(research_agent)

🔍 Research Agent
✅ Output:
 To find three key references and summarize them, I will utilize the Arxiv tool for academic papers, the Tavily tool for general web searches, and the Wikipedia tool for encyclopedic knowledge. 

Let's move forward with using these tools to gather the necessary information in parallel.
```json
{
  "tool_uses": [
    {
      "recipient_name": "functions.arxiv_tool",
      "parameters": {
        "query": "Key References in AI Research",
        "max_results": 3
      }
    },
    {
      "recipient_name": "functions.tavily_tool",
      "parameters": {
        "query": "important AI advancements",
        "max_results": 3
      }
    },
    {
      "recipient_name": "functions.wikipedia_tool",
      "parameters": {
        "query": "History of Artificial Intelligence"
      }
    }
  ]
}
```
🔍 Research Agent
✅ Output:
 To accomplish the task of briefly summarizing two seminal papers, I will utilize the available tools: arxiv_tool to search for academic papers, 

## Exercise 3: writer_agent

### Objective
Set up a call to a language model (LLM) for executing writing tasks like drafting, expanding, or summarizing text.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of a writing agent focused on generating academic or technical content.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

### Notes

- The function is designed to produce well-structured text by setting the correct prompts.
- Temperature is set to 1.0 to allow for creative variance in the writing outputs.

Ensure the system prompt and messages are defined properly to achieve a structured output from the LLM.

In [ ]:
# writer_agent
def writer_agent(task: str, model: str = "openai:gpt-4o") -> str:
    """
    Executes writing tasks, such as drafting, expanding, or summarizing text.
    """
    print("==================================")
    print("✍️ Writer Agent")
    print("==================================")

    ### START CODE HERE ###

    # Create the system prompt.
    # This should assign the LLM the role of a writing agent specialized in generating well-structured academic or technical content
    system_prompt = """
    You are a writing agent focused on generating academic or technical content.
    """

    # Define the system msg by using the system_prompt and assigning the role of system
    system_msg = {"role": "system" , "content": system_prompt}

    # Define the user msg. In this case the user prompt should be the task passed to the function
    user_msg = {"role": "user" , "content": task}

    # Add both system and user messages to the messages list
    messages = [system_msg, user_msg]

    response = CLIENT.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1.0
    )

    return response.choices[0].message.content

In [ ]:
# Test your code!
unittests.test_writer_agent(writer_agent)

✍️ Writer Agent
 All tests passed!


## Editor_agent

### Objective
Configure a call to a language model (LLM) to perform editorial tasks such as reflecting, critiquing, or revising drafts.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of an editor agent whose task is to reflect on, critique, or improve drafts.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

### Notes

- The editor agent is tailored for enhancing the quality of text by setting an appropriate role and task in the prompts.
- Temperature is set to 0.7, balancing creativity and coherence in editorial outputs.

Ensure the system prompt and messages are accurately set up to perform effective editorial tasks with the LLM.

In [ ]:
# editor_agent
def editor_agent(task: str, model: str = "openai:gpt-4o") -> str:
    """
    Executes editorial tasks such as reflection, critique, or revision.
    """
    print("==================================")
    print("🧠 Editor Agent")
    print("==================================")


    # Create the system prompt.
    # This should assign the LLM the role of an editor agent specialized in reflecting on, critiquing, or improving existing drafts.
    system_prompt = f"""
            You are an editor agent, specialized in reflecting on, critiquing, or improving existing drafts.
    """

    # Define the system msg by using the system_prompt and assigning the role of system
    system_msg = {"role": "system" , "content": system_prompt}

    # Define the user msg. In this case the user prompt should be the task passed to the function
    user_msg = {"role": "user" , "content": task}

    # Add both system and user messages to the messages list
    messages = [system_msg, user_msg]


    response = CLIENT.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7
    )

    return response.choices[0].message.content

In [ ]:
# Test your code!
unittests.test_editor_agent(editor_agent)

🧠 Editor Agent
 All tests passed!


### 🎯 The Executor Agent

The `executor_agent` manages the workflow by executing each step of a given plan. It:

1. Decides **which agent** (`research_agent`, `writer_agent`, or `editor_agent`) should handle the step.
2. Builds context from the outputs of previous steps.
3. Sends the enriched task to the selected agent.
4. Collects and stores the results in a shared history.

👉 **Do not implement or modify this function.** It is already provided as the orchestration component of the multi-agent pipeline.

Notice that `planner_agent` might return a long list of steps. Because of this, the maximum number of steps is set to a maximum of 4 to keep running time reasonable.

In [ ]:
agent_registry = {
    "research_agent": research_agent,
    "editor_agent": editor_agent,
    "writer_agent": writer_agent,
}

def clean_json_block(raw: str) -> str:
    """
    Clean the contents of a JSON block that may come wrapped with Markdown backticks.
    """
    raw = raw.strip()
    if raw.startswith("```"):
        raw = re.sub(r"^```(?:json)?\n?", "", raw)
        raw = re.sub(r"\n?```$", "", raw)
    return raw.strip()

In [ ]:
def executor_agent(topic, model: str = "openai:gpt-4o", limit_steps: bool = True):

    plan_steps = planner_agent(topic)
    max_steps = 4

    if limit_steps:
        plan_steps = plan_steps[:min(len(plan_steps), max_steps)]

    history = []

    print("==================================")
    print("🎯 Editor Agent")
    print("==================================")

    for i, step in enumerate(plan_steps):

        agent_decision_prompt = f"""
        You are an execution manager for a multi-agent research team.

        Given the following instruction, identify which agent should perform it and extract the clean task.

        Return only a valid JSON object with two keys:
        - "agent": one of ["research_agent", "editor_agent", "writer_agent"]
        - "task": a string with the instruction that the agent should follow

        Only respond with a valid JSON object. Do not include explanations or markdown formatting.

        Instruction: "{step}"
        """
        response = CLIENT.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": agent_decision_prompt}],
            temperature=0,
        )

        raw_content = response.choices[0].message.content
        cleaned_json = clean_json_block(raw_content)
        agent_info = json.loads(cleaned_json)

        agent_name = agent_info["agent"]
        task = agent_info["task"]

        context = "\n".join([
            f"Step {j+1} executed by {a}:\n{r}"
            for j, (s, a, r) in enumerate(history)
        ])
        enriched_task = f"""
        You are {agent_name}.

        Here is the context of what has been done so far:
        {context}

        Your next task is:
        {task}
        """

        print(f"\n🛠️ Executing with agent: `{agent_name}` on task: {task}")

        if agent_name in agent_registry:
            output = agent_registry[agent_name](enriched_task)
            history.append((step, agent_name, output))
        else:
            output = f"⚠️ Unknown agent: {agent_name}"
            history.append((step, agent_name, output))

        print(f"✅ Output:\n{output}")

    return history

In [ ]:
# If you want to see the full workflow without limiting the number of steps. Set limit_steps to False
# Keep in mind this could take more than 10 minutes to complete
executor_history = executor_agent("The ensemble Kalman filter for time series forecasting", limit_steps=True)

md = executor_history[-1][-1].strip("`")
display(Markdown(md))

🎯 Editor Agent

🛠️ Executing with agent: `research_agent` on task: Search Wikipedia and textbooks for fundamental definitions and the mathematical formulation of the ensemble Kalman filter
🔍 Research Agent
✅ Output:
 To accomplish this task, I'll use the Wikipedia tool to obtain the fundamental definitions and mathematical formulation of the Ensemble Kalman Filter.  After gathering information from Wikipedia, I'll summarize the findings and provide the current date for context. Let's begin by searching Wikipedia for this information.
multi_tool_use.parallel ({_()
tool_uses: {_
{
recipient_name: "wikipedia_tool",
parameters: {_
query: "Ensemble Kalman Filter"
}_,
}_,
},
}_);
✅ Output:
To accomplish this task, I'll use the Wikipedia tool to obtain the fundamental definitions and mathematical formulation of the Ensemble Kalman Filter.  After gathering information from Wikipedia, I'll summarize the findings and provide the current date for context. Let's begin by searching Wikipedia for th

# Introduction

## Background

Time series forecasting is a critical task in various scientific and industrial fields, ranging from meteorology and finance to control systems and bioinformatics. The primary goal of time series forecasting is to predict future values of a sequence based on past observed data, offering invaluable insights for decision-making and strategic planning. However, the inherent uncertainties and noise in real-world data present significant challenges that necessitate sophisticated forecasting methods.

The Ensemble Kalman Filter (EnKF) is a prominent data assimilation algorithm originally developed for nonlinear systems with considerable model error and uncertainty, particularly within meteorological and oceanographic contexts. Introduced as an extension of the classic Kalman Filter, the EnKF utilizes a Monte Carlo approach to address the limitations posed by linear assumptions and Gaussian noise in highly complex systems. By employing a set of ensemble forecasts rather than a single deterministic model, the EnKF captures a diverse range of possible future states, thereby enhancing the reliability of its predictions.

## Motivation

The adaptation of the Ensemble Kalman Filter to time series forecasting emerges from a growing need for robust methods that can manage the dynamic and often chaotic nature of real-world time series data. Traditional statistical methods may falter under the weight of non-linearity and non-stationarity, common characteristics of time series data generated by complex systems. Meanwhile, the ability of the EnKF to iteratively update forecasts by integrating real-time observational data offers a promising avenue to mitigate these challenges.

Moreover, the EnKF's capability of assimilating data through an ensemble of model states allows for a comprehensive representation of the uncertainty associated with forecasts. This makes it particularly well-suited to applications where capturing the uncertainty is as critical as obtaining precise predictions, such as in weather forecasting or risk assessment in finance.

## Objectives

The primary objective of applying the Ensemble Kalman Filter in time series forecasting is to enhance predictive accuracy and manage uncertainty more effectively than traditional approaches. By leveraging the adaptive and recursive properties of the EnKF, researchers and practitioners aim to refine forecasts continually as new data becomes available, achieving a balance between computational feasibility and forecasting precision.

Key research endeavors focus on optimizing the implementation of the EnKF for time series data by fine-tuning model parameters, developing hybrid models that couple EnKF with other statistical or machine learning techniques, and exploring its applicability across diverse domains. The ongoing exploration into these areas seeks to unlock the full potential of the EnKF, solidifying its role as a cornerstone methodology in the arsenal of modern forecasting tools.